In [1]:
import cupy as cp 
import numpy as np
import pygraphblas as gb

# Check if connected to GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU: need to configure runtime type to an environment with an accessible GPU')
else:
    print('Connected to a GPU - GPU info summary: \n\n' + gpu_info)
    
# Use python wrapper of GraphBLAS on GPU (BLAS - Basic Linear Algebra Subprograms)
# GraphBLAS supports graph operations via linear algebraic methods (e.g. matrix multiplication) over various semirings

# GraphBLAS version of BMLP-RMS algorithm which performs repeated matrix squaring
from bmlp import Matrix, Operator, Predicate, Generator, Task

Connected to a GPU - GPU info summary: 

Tue Nov 19 16:41:59 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.58.02              Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:47:00.0  On |                  N/A |
| 31%   27C    P8              8W /  285W |     723MiB /  16376MiB |      3%      Default |
|                                         |                        |                  N/A |
+------

In [2]:
# Create a sequence of boolean matrix operations to compute
#   p0(X1, X2) :- p1(X1, X3), p2(X3, X4), p3(X4, X2).
num_nodes = 5
p1 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p2 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p3 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)

p1[0,1] = True
print('p1 = \n' + str(p1) + '\n')

p2[1,2] = True
print('p2 = \n' + str(p2) + '\n')

p3[2,3] = True
print('p3 = \n' + str(p3) + '\n')

# exactly-two-connected program in chained H2m
p0 = p1 @ p2 @ p3
print('p0 = \n' + str(p0) + '\n')

p1 = 
      0  1  2  3  4
  0|     t         |  0
  1|               |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p2 = 
      0  1  2  3  4
  0|               |  0
  1|        t      |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p3 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|           t   |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p0 = 
      0  1  2  3  4
  0|           t   |  0
  1|               |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4



In [3]:
# Create a sequence of boolean matrix operations to compute
#   p0(X1, X2) :- p1(X2, X1).
#   p0(X1, X2) :- p2(X1, X3), p3(X3, X4), p0(X4, X2).
num_nodes = 5
p1 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p2 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p3 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)

p1[1,0] = True
print('p1 = \n' + str(p1) + '\n')

p2[1,2] = True
p2[3,4] = True
print('p2 = \n' + str(p2) + '\n')

p3[2,3] = True
p3[4,0] = True
print('p3 = \n' + str(p3) + '\n')

print('p2 x p3 = \n' + str(p2 @ p3) + '\n')

# exactly-two-connected recursion in chained H2m
p0 = Matrix.BMLP_RMS(p1.T,p2 @ p3,print_matrix=True)
print('p0 = \n' + str(p0) + '\n')

p1 = 
      0  1  2  3  4
  0|               |  0
  1|  t            |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p2 = 
      0  1  2  3  4
  0|               |  0
  1|        t      |  1
  2|               |  2
  3|              t|  3
  4|               |  4
      0  1  2  3  4

p3 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|           t   |  2
  3|               |  3
  4|  t            |  4
      0  1  2  3  4

p2 x p3 = 
      0  1  2  3  4
  0|               |  0
  1|           t   |  1
  2|               |  2
  3|  t            |  3
  4|               |  4
      0  1  2  3  4

R = R2 + I = 
      0  1  2  3  4
  0|  t            |  0
  1|     t     t   |  1
  2|        t      |  2
  3|  t        t   |  3
  4|              t|  4
      0  1  2  3  4

fixpoint = 
      0  1  2  3  4
  0|  t            |  0
  1|  t  t     t   |  1
  2|        t      |  2
  3|  t        t   |  3
  4|              t|  4


In [4]:
# Create a sequence of boolean matrix operations to compute
#   p0(X1, X2) :- p1(X1, X3), p2(X1, X4), p3(X2, X3), p4(X2, X4), p5(X3, X4).
# => X1 - X3 
#       X  | 
# <= X2 - X4

# C1
# => X1 - X3 - X4 - X2 =>
# C2
# => X1 - X4 - X3 - X2 =>

# p1 @ p5 @ p4.T /\ C2

# x, +, ~, T, /\

num_nodes = 5
p1 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p2 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p3 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p4 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)
p5 = gb.Matrix.sparse(gb.BOOL, num_nodes, num_nodes)

p1[1,3] = True
print('p1 = \n' + str(p1) + '\n')

p2[1,4] = True
print('p2 = \n' + str(p2) + '\n')

p3[2,3] = True
print('p3 = \n' + str(p3) + '\n')

p4[2,4] = True
print('p4 = \n' + str(p4) + '\n')

p5[3,4] = True
print('p5 = \n' + str(p5) + '\n')

p0 = p1 @ p5 @ p4.T * (p2 @ p5.T @ p3.T)
print('p0 = \n' + str(p0) + '\n')

p1 = 
      0  1  2  3  4
  0|               |  0
  1|           t   |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p2 = 
      0  1  2  3  4
  0|               |  0
  1|              t|  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p3 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|           t   |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p4 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|              t|  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

p5 = 
      0  1  2  3  4
  0|               |  0
  1|               |  1
  2|               |  2
  3|              t|  3
  4|               |  4
      0  1  2  3  4

p0 = 
      0  1  2  3  4
  0|               |  0
  1|        t      |  1
  2|               |  2
  3|               |  3
  4|               |  4
      0  1  2  3  4

In [ ]:
#######################################################
# Background knowledge:
# 
#			    harry+sally
#	   	        /		\
#		    john		mary
# 
# father(harry,john). mother(sally,john). 
# father(harry,mary). mother(sally,mary).
# male(harry). female(sally).
# male(john). female(mary).
# 
# Constant to matrix index mapping:
#   (0, harry), (1, john), (2, mary), (3, sally) 

# Background knowledge encoded by matrices
m1 = gb.Matrix.sparse(gb.BOOL, 64, 64)
m1[0, 1] = True
m1[0, 2] = True
father = Predicate.new_predicate(m1, "father")

m2 = gb.Matrix.sparse(gb.BOOL, 64, 64)
m2[3, 1] = True
m2[3, 2] = True
mother = Predicate.new_predicate(m2, "mother")

m3 = gb.Matrix.sparse(gb.BOOL, 64, 64)
m3[0, 0] = True
m3[1, 1] = True
male = Predicate.new_predicate(m3, "male")

m4 = gb.Matrix.sparse(gb.BOOL, 64, 64)
m4[2, 2] = True
m4[3, 3] = True
female = Predicate.new_predicate(m4, "female")

In [ ]:
# Target:
#   parent
# 
# Examples:
#   E+ = 
#       {   parent(harry,john). parent(sally,john). 
#           parent(harry,mary). parent(sally,mary). }
#   E- = 
#       {   parent(harry,sally). parent(mary,john).  
#           parent(harry,harry).                    }
# Postive examples
pos = gb.Matrix.sparse(gb.BOOL, 64, 64)
pos[0, 1] = True
pos[0, 2] = True
pos[3, 1] = True
pos[3, 2] = True
# Negative examples
neg = gb.Matrix.sparse(gb.BOOL, 64, 64)
neg[0, 3] = True
neg[2, 1] = True
neg[0, 0] = True

learn_parent = Task.new_task(pos, neg)

print(f'No. pos example: {learn_parent.get_pn()}')
print(f'No. neg example: {learn_parent.get_nn()}')

# Primitive predicates are the depth 1 predicates
primitives = [father, mother, male, female]
# Define a generator using default operators
generator = Generator.Generator(primitives)
success, res, pred = generator.invent_predicates_for_task(learn_parent)

if success:
    print('Primitives invented successfully')
    print(res)


No. pos example: 4
No. neg example: 3
New Pred:  inv_14(X, Y) :- father(X, Y).
inv_14(X, Y) :- mother(X, Y).

Primitives invented successfully
[Predicate(
matrix=
      0  1  2  3
  0|     t  t   |  0
  1|            |  1
  2|            |  2
  3|     t  t   |  3
      0  1  2  3, 
name=inv_14, scores=(1.0, 0.0))]


In [ ]:
#######################################################
# Background knowledge:
# 
#			    harry+sally
#	   	        /		\
#		    john		mary
#             |          |
#           bill        maggie       
# father(harry,john). mother(sally,john). 
# father(harry,mary). mother(sally,mary).
# father(john, bill). mother(mary, maggie).
# male(harry). female(sally).
# male(john). female(mary).
# male(bill). female(maggie).
# 
# Constant to matrix index mapping:
#   (0, harry), (1, john), (2, mary), (3, sally), (4, bill), (5, maggie)
# 
# 
# Target:
#   grandparent
# 
# Examples:
#   E+ = 
#       {   grandparent(harry,bill). grandparent(sally,bill). 
#           grandparent(harry,maggie). grandparent(sally,maggie). }
#   E- = 
#       {   grandparent(harry,sally). grandparent(mary,john).  
#           grandparent(harry,harry). grandparent(john, bill).  }

father.get_matrix()[1, 4] = True
mother.get_matrix()[2, 5] = True
male.get_matrix()[4, 4] = True
female.get_matrix()[5, 5] = True

# Postive examples
pos[0, 4] = True
pos[0, 5] = True
pos[3, 4] = True
pos[3, 5] = True

# Negative examples
neg[0, 3] = True
neg[2, 1] = True
neg[0, 0] = True
neg[1, 4] = True

learn_grandparent = Task.new_task(pos, neg)
print(f'No. pos example: {learn_grandparent.get_pn()}')
print(f'No. neg example: {learn_grandparent.get_nn()}')

primitives = [father, mother, male, female]
generator = Generator.Generator(primitives)
cached = {}

success, _, iter_2 = generator.invent_predicates_for_task(learn_grandparent, cached)
print(f'Target: {success}, Size 2 programs: {len(iter_2)}, Redundant programs: {generator.get_redundancy()}')

generator.update_predicates(iter_2 + primitives)
success, res, iter_3 = generator.invent_predicates_for_task(learn_grandparent, cached)
print(f'Target: {success}, Size 3 programs: {len(iter_3)}, Redundant programs: {generator.get_redundancy()}')

generator.update_predicates(iter_3 + iter_2 + primitives)
success, _, iter_4 = generator.invent_predicates_for_task(learn_grandparent, cached)
print(f'Target: {success}, Size 4 programs: {len(iter_4)}, Redundant programs: {generator.get_redundancy()}')

generator.update_predicates(iter_4 + iter_3 + iter_2 + primitives)
success, _, iter_5 = generator.invent_predicates_for_task(learn_grandparent, cached)
print(f'Target: {success}, Size 5 programs: {len(iter_5)}, Redundant programs: {generator.get_redundancy()}')

generator.update_predicates(iter_5 + iter_4 + iter_3 + iter_2 + primitives)
success, _, iter_6 = generator.invent_predicates_for_task(learn_grandparent, cached)
print(f'Target: {success}, Size 6 programs: {len(iter_6)}, Redundant programs: {generator.get_redundancy()}')

print(res[0])

No. pos example: 4
No. neg example: 4
New Pred:  inv_14(X, Y) :- father(X, Y).
inv_14(X, Y) :- mother(X, Y).

redundant:       0  1  2  3  4  5
  0|  t               |  0
  1|     t            |  1
  2|                 t|  2
  3|     t  t         |  3
  4|              t   |  4
  5|                  |  5
      0  1  2  3  4  5
Target: False, Size 2 programs: 17, Redundant programs: 31
New Pred:  inv_14(X, Y) :- father(X, Y).
inv_14(X, Y) :- mother(X, Y).
inv_76(X, Y) :- inv_14(Y, X).

New Pred:  inv_14(X, Y) :- father(X, Y).
inv_14(X, Y) :- mother(X, Y).
inv_77(X, Y) :- inv_14(X, Y).
inv_77(X, Y) :- inv_14(X, Z), inv_77(Z, Y).

New Pred:  inv_14(X, Y) :- father(X, Y).
inv_14(X, Y) :- mother(X, Y).
inv_78(X, Y) :- inv_14(X, Z), inv_14(Z, Y).

redundant:       0  1  2  3  4  5
  0|  t               |  0
  1|     t            |  1
  2|                 t|  2
  3|     t  t        t|  3
  4|              t   |  4
  5|                  |  5
      0  1  2  3  4  5
redundant:       0  1  2  3  

In [ ]:
#######################################################
# Background knowledge:
# 
#			    harry+sally
#	   	        /		\
#		    john		mary
#             |          |
#           bill        maggie
#           \  
#           ted             
# 
# father(harry,john). mother(sally,john). 
# father(harry,mary). mother(sally,mary).
# father(john, bill). mother(mary, maggie).
# father(bill, ted).
# male(harry). female(sally).
# male(john). female(mary).
# male(bill). female(maggie).
# male(ted).
# 
# Constant to matrix index mapping:
#   (0, harry), (1, john), (2, mary), (3, sally), (4, bill), (5, maggie), (6, ted)
# 
# 
# Target:
#   ancestor
# 
# Examples:
#   E+ = 
#       {   
#           ancestor(harry,ted).
#           ancestor(harry,bill). ancestor(sally,bill). 
#           ancestor(harry,maggie). ancestor(sally,maggie). }
#   E- = 
#       {   ancestor(harry,sally). ancestor(mary,john).  
#           ancestor(harry,harry). ancestor(ted, bill).  }

father.get_matrix()[1, 4] = True
father.get_matrix()[4, 6] = True
mother.get_matrix()[2, 5] = True
male.get_matrix()[4, 4] = True
female.get_matrix()[5, 5] = True

# Postive examples
pos[0, 4] = True
pos[0, 5] = True
pos[0, 6] = True
pos[3, 4] = True
pos[3, 5] = True

# Negative examples
neg[0, 3] = True
neg[2, 1] = True
neg[0, 0] = True
neg[6, 4] = True

learn_grandparent = Task.new_task(pos, neg)
print(f'No. pos example: {learn_grandparent.get_pn()}')
print(f'No. neg example: {learn_grandparent.get_nn()}')

primitives = [father, mother, male, female]
generator = Generator.Generator(primitives)
cached = {}

success, _, iter_2 = generator.invent_predicates_for_task(learn_grandparent, cached)
print(f'Target: {success}, Size 2 programs: {len(iter_2)}, Redundant programs: {generator.get_redundancy()}')

generator.update_predicates(iter_2 + primitives)
success, _, iter_3 = generator.invent_predicates_for_task(learn_grandparent, cached)
print(f'Target: {success}, Size 3 programs: {len(iter_3)}, Redundant programs: {generator.get_redundancy()}')

# generator.update_predicates(iter_3 + iter_2 + primitives)
# success, _, iter_4 = generator.invent_predicates_for_task(learn_grandparent, cached)
# print(f'Target: {success}, Size 4 programs: {len(iter_4)}, Redundant programs: {generator.get_redundancy()}')

# generator.update_predicates(iter_4 + iter_3 + iter_2 + primitives)
# success, _, iter_5 = generator.invent_predicates_for_task(learn_grandparent, cached)
# print(f'Target: {success}, Size 5 programs: {len(iter_5)}, Redundant programs: {generator.get_redundancy()}')

# generator.update_predicates(iter_5 + iter_4 + iter_3 + iter_2 + primitives)
# success, _, iter_6 = generator.invent_predicates_for_task(learn_grandparent, cached)
# print(f'Target: {success}, Size 6 programs: {len(iter_6)}, Redundant programs: {generator.get_redundancy()}')

# generator.update_predicates(iter_6 + iter_5 + iter_4 + iter_3 + iter_2 + primitives)
# success, _, iter_7 = generator.invent_predicates_for_task(learn_grandparent, cached)
# print(f'Target: {success}, Size 6 programs: {len(iter_7)}, Redundant programs: {generator.get_redundancy()}')

#      0  1  2  3  4  5  6
#   0|     t  t     t  t  t|  0
#   1|              t     t|  1
#   2|                 t   |  2
#   3|     t  t     t  t  t|  3
#   4|                    t|  4
#   5|                     |  5
#   6|                     |  6
#       0  1  2  3  4  5  6

#      0  1  2  3  4  5  6
#   0|     t  t            |  0
#   1|              t      |  1
#   2|                 t   |  2
#   3|     t  t            |  3
#   4|                    t|  4
#   5|                     |  5
#   6|                     |  6
#       0  1  2  3  4  5  6


No. pos example: 5
No. neg example: 4
redundant:       0  1  2  3  4  5  6
  0|     t  t            |  0
  1|              t      |  1
  2|                 t   |  2
  3|     t  t            |  3
  4|                    t|  4
  5|                     |  5
  6|                     |  6
      0  1  2  3  4  5  6
New Pred:  inv_14(X, Y) :- father(X, Y).
inv_14(X, Y) :- mother(X, Y).

redundant:       0  1  2  3  4  5  6
  0|  t                  |  0
  1|     t               |  1
  2|                 t   |  2
  3|     t  t            |  3
  4|              t      |  4
  5|                     |  5
  6|                     |  6
      0  1  2  3  4  5  6
Target: False, Size 2 programs: 14, Redundant programs: 34
New Pred:  inv_14(X, Y) :- father(X, Y).
inv_14(X, Y) :- mother(X, Y).
inv_73(X, Y) :- inv_14(Y, X).

New Pred:  inv_14(X, Y) :- father(X, Y).
inv_14(X, Y) :- mother(X, Y).
inv_74(X, Y) :- inv_14(X, Y).
inv_74(X, Y) :- inv_14(X, Z), inv_74(Z, Y).

redundant:       0  1  2  3  4  5  6
